# Regresión Logística: Manejo de Datos Faltantes

## 1. Introducción

La Regresión Logística es un modelo de clasificación que estima probabilidades. En este proyecto, trabajaremos con el dataset *Horse Colic* para predecir la supervivencia de caballos basándonos en datos médicos.

**Reto:** El dataset tiene muchos valores perdidos (`?`). Compararemos tres estrategias para solucionarlo:
1.  **Cero (Zero):** Reemplazar `?` con 0.
2.  **Media (Mean):** Reemplazar con el promedio de la columna.
3.  **Mediana (Median):** Reemplazar con el valor central (robusto a outliers).

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Configuración
pd.set_option('display.max_columns', None)

## 2. Implementación de Estrategias de Imputación

Creamos una función flexible que carga y limpia los datos según la estrategia elegida.

In [12]:
def load_and_process(path, strategy='zero'):
    """
    Carga, limpia y transforma los datos.
    Estrategias: 'zero', 'mean', 'median'.
    """
    try:
        # Cargar datos tratando '?' como NaN
        data = pd.read_csv(path, sep=r'\s+', header=None, na_values='?', engine='python')
    except FileNotFoundError:
        return None, None

    # Target (Columna 22): 1=Vivo, 2=Muerto, 3=Sacrificado
    # Transformamos a Binario: 1=Vivo, 2=No Vivo (Muerto/Sacrificado)
    label = data[22].copy()
    label.replace({3: 2}, inplace=True)
    
    # Limpiar filas con target nulo
    mask = label.notna()
    data = data[mask]
    label = label[mask]
    
    # Features (eliminar target y datos extra)
    features = data.drop(columns=range(22, 28)).astype(float)
    
    # --- APLICAR ESTRATEGIA ---
    if strategy == 'zero':
        features = features.fillna(0)
        
    elif strategy == 'mean':
        for col in features.columns:
            features[col] = features[col].fillna(features[col].mean())
            
    elif strategy == 'median':
        for col in features.columns:
            features[col] = features[col].fillna(features[col].median())
            
    return features, label.astype(int)

## 3. Comparación de Resultados

Entrenamos un modelo de Regresión Logística para cada caso y comparamos la exactitud (*Accuracy*).

In [13]:
# Modelo Base
clf = LogisticRegression(random_state=0, max_iter=2000)
strategies = ['zero', 'mean', 'median']

print(f"{'Estrategia':<15} | {'Accuracy':<10}")
print("-" * 30)
scaler = StandardScaler()
try:
    for strat in strategies:
        # 1. Cargar datos
        X_train, y_train = load_and_process("../data/horse-colic.data", strategy=strat)
        X_test, y_test = load_and_process("../data/horse-colic.test", strategy=strat)

        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        if X_train is not None:
            # 2. Entrenar
            clf.fit(X_train_scaled, y_train)
            
            # 3. Evaluar
            pred = clf.predict(X_test_scaled)
            acc = accuracy_score(y_test, pred)
            
            print(f"{strat.capitalize():<15} | {acc:.4f}")
        else:
            print("Archivos de datos no encontrados.")
            break

except Exception as e:
    print(f"Error: {e}")

Estrategia      | Accuracy  
------------------------------
Zero            | 0.7164
Mean            | 0.7761
Median          | 0.7612


### Análisis y conclusión
Al escalar los datos, el modelo logró converger y los resultados se alinearon con la teoría estadística. La estrategia de imputación por Media obtuvo el mejor desempeño (77.6%), demostrando que en este conjunto de datos clínicos, preservar la tendencia central de las variables es más efectivo que asumir valores nulos (Cero).

## 4. Pregunta Teórica: Clasificación Multiclase

**Problema:** Si tuviéramos 3 clases (Rojo, Verde, Azul) en lugar de 2, ¿cómo adaptamos la Regresión Logística?

**Solución:** Usando la heurística **One-vs-All (Uno contra Todos)**.
1.  Se divide el problema en 3 sub-problemas binarios:
    * Rojo vs (Verde + Azul)
    * Verde vs (Rojo + Azul)
    * Azul vs (Rojo + Verde)
2.  Se entrena un modelo para cada uno.
3.  Al predecir, se elige la clase del modelo que arroje la probabilidad más alta.